# IGRINS CA1 interactive groove schematic *with laser*

Here we make an interactive schematic of the IGRINS CA-1 groove, now with a laser

In [1]:
import matplotlib.pyplot as plt
import numpy as np

%config InlineBackend.figure_format='retina'

### Fixed physical properties

In [2]:
sigma = 27.36 # micron

In [3]:
groove_top_width = 9.95 # micron

Here is a cartoon of the IGRINS CA-1 grating from [this slideshow](https://speakerdeck.com/gully/ideas-for-metrology-of-silicon-diffractive-optics?slide=45).

In [4]:
%%html
<img src="../IGRINS_CA1_schematic.png" width=300></img>

The angle $a$ is the fixed $70.53^\circ$ from the Si crystal planes, $b$ is $\arctan{3}$, and c is whatever is left over to add up to 180.

In [5]:
a_degrees = 70.53 # "Fixed" Si crystal plane angle

In [6]:
b_degrees = np.degrees(np.arctan(3)) # Blaze angle

## Create the function

In [7]:
def grating_geometry(sigma, groove_top_width, b_degrees, a_degrees):
    """Compute the grating groove geometry from math
    
    Parameters:
    ----------
    
    sigma (micron): 
        grating groove spacing or "groove constant"
    groove_top_width (micron):
        Si grating groove top width defined by UV photomask chrome stripes
    b_degrees (degrees):
        The b angle from Figure 1 above, often the blaze angle
    a_degrees (degrees):
        The a angle from Figure 1 above, also called the Si apex angle
    """
    
    c_degrees = 180 - a_degrees - b_degrees # Complement must add up to 180
    
    a_radians, b_radians, c_radians = (np.radians(angle) for angle in 
                                   [a_degrees, b_degrees, c_degrees])
    
    # Compute the line segments A, B, C with the Law of Sines:
    A_micron = sigma - groove_top_width
    
    fixed_ratio = np.sin(a_radians) / A_micron
    B_micron = np.sin(b_radians) / fixed_ratio
    C_micron = np.sin(c_radians) / fixed_ratio
    
    # Compute the vertices, define vertex `c` as the origin
    c_vertex = np.array([0.0, 0.0])
    b_vertex = c_vertex + np.array([A_micron, 0.0])
    a_vertex = np.array([B_micron*np.cos(c_radians), -B_micron*np.sin(c_radians)])
    
    # Define one other key point, the coordinate of the start of the groove top.
    d_vertex = np.array([-groove_top_width, 0.0])
    
    return {'angle_degrees':(a_degrees, b_degrees, c_degrees),
            'angle_radians':(a_radians, b_radians, c_radians),
            'segments_micron':(A_micron, B_micron, C_micron),
           'vertices_micron':(a_vertex, b_vertex, c_vertex, d_vertex),
           'B_x_coordinates': [c_vertex[0], a_vertex[0]],
           'B_y_coordinates': [c_vertex[1], a_vertex[1]],
            'C_x_coordinates':[a_vertex[0], b_vertex[0]], 
            'C_y_coordinates':[a_vertex[1], b_vertex[1]],
            'D_x_coordinates':[c_vertex[0], d_vertex[0]], 
            'D_y_coordinates':[c_vertex[1], d_vertex[1]]
           }

In [8]:
geometry_dict = grating_geometry(sigma, groove_top_width, b_degrees, a_degrees)

## Make a function for geometry of laser incident on the groove

In [9]:
def laser_geometry(sigma, groove_top_width, b_degrees, a_degrees, incidence_angle):
    """Compute the laser geometry for groove properties and incidence angle
    
    Parameters:
    ----------
    (all the same as above, plus:)
    
    incidence angle (degrees):
        The laser's angle of incidence on the grating as measured from the
        grating normal.  Positive is measured counter-clockwise from the normal,
        negative is measured counter-clockwise from the normal.  The angle is 
        defined between -90 and 90 degrees.
    """
    geometry_dict = grating_geometry(sigma, groove_top_width, b_degrees, a_degrees)

    incidence_angle_radians = np.radians(incidence_angle)
    
    gamma_degrees = 90 - incidence_angle
    gamma_radians = np.radians(gamma_degrees)
    
    
    phi_degrees = 180 - geometry_dict['angle_degrees'][1] - gamma_degrees
    phi_radians = np.radians(phi_degrees)
    
    h_micron = geometry_dict['segments_micron'][0] * np.sin(gamma_radians)/np.sin(phi_radians)
    
    if h_micron > geometry_dict['segments_micron'][2]:
        h_micron = geometry_dict['segments_micron'][2]
        
    b_radians = geometry_dict['angle_radians'][1]
    h_vertex = np.array([geometry_dict['segments_micron'][0] - h_micron*np.cos(b_radians),
                         -h_micron*np.sin(b_radians)])
    
    
        
    return {#'h_micron':h_micron, 'h_vertex':h_vertex,
            'h_x_coordinates':[h_vertex[0], geometry_dict['vertices_micron'][1][0]], 
            'h_y_coordinates':[h_vertex[1], geometry_dict['vertices_micron'][1][1]]}

In [10]:
laser_input = laser_geometry(sigma, groove_top_width, b_degrees, a_degrees, 70.0)

In [11]:
bokeh_input = {key:np.array(geometry_dict[key]) for key in 
               ['B_x_coordinates', 'B_y_coordinates', 
                'C_x_coordinates', 'C_y_coordinates',
               'D_x_coordinates', 'D_y_coordinates']}

## Add a bokeh interactive dashboard

In [12]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Slider, Span, Range1d, Dropdown
from bokeh.layouts import layout, Spacer
from bokeh.models.widgets import Button, Div

In [13]:
bokeh_input

{'B_x_coordinates': array([ 0.        , 13.82251582]),
 'B_y_coordinates': array([  0.        , -10.76245254]),
 'C_x_coordinates': array([13.82251582, 17.41      ]),
 'C_y_coordinates': array([-10.76245254,   0.        ]),
 'D_x_coordinates': array([ 0.  , -9.95]),
 'D_y_coordinates': array([0., 0.])}

In [14]:
def create_interact_ui(doc):

    # Make the spectrum source

    data_source = ColumnDataSource(data=bokeh_input)
    laser_source = ColumnDataSource(data=laser_input)

    fig = figure(
        title="Si grating groove geometry",
        plot_height=500,
        plot_width=500,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location="below",
        border_fill_color="whitesmoke",
        #match_aspect=True
    )
    fig.title.offset = -10
    fig.yaxis.axis_label = "y (micron)"
    fig.xaxis.axis_label = "x (micron)"
    fig.y_range = Range1d(start=-30, end=20)

    fig.line(
        "B_x_coordinates",
        "B_y_coordinates",
        line_width=2,
        legend_label="Shallow Facet",
        color='mediumslateblue',
        source=data_source,
    )
    
    fig.line(
        "h_x_coordinates",
        "h_y_coordinates",
        line_width=4,
        legend_label="Shallow Facet",
        color='green',
        source=laser_source,
    )

    fig.x_range = Range1d(start=-20, end=30)

    fig.line(
        "C_x_coordinates",
        "C_y_coordinates",
        line_width=2,
        legend_label="Scarp Facet",
        color='orangered',
        source=data_source,
    )
    
    fig.line(
        "D_x_coordinates",
        "D_y_coordinates",
        line_width=2,
        color='goldenrod',
        legend_label="Groove Top",
        source=data_source,
    )

    fig.legend.location = "top_left"
    fig.legend.orientation = "horizontal"

    # Slider to decimate the data
    blaze_slider = Slider(
        start=65.0,
        end=78.0,
        value=71.5,
        step=0.05,
        title="Blaze Angle (Degrees)",
        width=490,
    )
    
    # Slider for sigma
    sigma_slider = Slider(
        start=20.0,
        end=35.0,
        value=27.36,
        step=0.05,
        title="Groove Pitch (micron)",
        width=490,
    )
    
    top_slider = Slider(
        start=1.0,
        end=19.0,
        value=9.95,
        step=0.05,
        title="Groove Top (Micron)",
        width=490,
    )
    
    apex_slider = Slider(
        start=69.0,
        end=73.0,
        value=70.53,
        step=0.03,
        title="Apex Angle (Degrees)",
        width=490,
    )
    
    incidence_slider = Slider(
        start=40.0,
        end=89.0,
        value=71.5,
        step=0.1,
        title="Incidence Angle (Degrees)",
        width=490,
    )

    def update_upon_incidence(attr, old, new):
        """Callback to take action when smoothing slider changes"""
        geometry_dict = grating_geometry(sigma_slider.value, top_slider.value, 
                                         blaze_slider.value, apex_slider.value)
        bokeh_input = {key:np.array(geometry_dict[key]) for key in 
                       ['B_x_coordinates', 'B_y_coordinates', 
                        'C_x_coordinates', 'C_y_coordinates',
                       'D_x_coordinates', 'D_y_coordinates']}
        data_source.data = bokeh_input
        laser_input = laser_geometry(sigma_slider.value, top_slider.value, 
                                         blaze_slider.value, apex_slider.value, new)
        laser_source.data = laser_input
    
    def update_upon_blaze(attr, old, new):
        """Callback to take action when smoothing slider changes"""
        geometry_dict = grating_geometry(sigma_slider.value, top_slider.value, 
                                         new, apex_slider.value)
        bokeh_input = {key:np.array(geometry_dict[key]) for key in 
                       ['B_x_coordinates', 'B_y_coordinates', 
                        'C_x_coordinates', 'C_y_coordinates',
                       'D_x_coordinates', 'D_y_coordinates']}
        data_source.data = bokeh_input
        
    def update_upon_sigma(attr, old, new):
        """Callback to take action when smoothing slider changes"""
        geometry_dict = grating_geometry(new, top_slider.value, blaze_slider.value, 
                                         apex_slider.value)
        bokeh_input = {key:np.array(geometry_dict[key]) for key in 
                       ['B_x_coordinates', 'B_y_coordinates', 
                        'C_x_coordinates', 'C_y_coordinates',
                       'D_x_coordinates', 'D_y_coordinates']}
        data_source.data = bokeh_input

    def update_upon_top(attr, old, new):
        """Callback to take action when smoothing slider changes"""
        geometry_dict = grating_geometry(sigma_slider.value, new, blaze_slider.value, 
                                         apex_slider.value)
        bokeh_input = {key:np.array(geometry_dict[key]) for key in 
                       ['B_x_coordinates', 'B_y_coordinates', 
                        'C_x_coordinates', 'C_y_coordinates',
                       'D_x_coordinates', 'D_y_coordinates']}
        data_source.data = bokeh_input
        
    def update_upon_apex(attr, old, new):
        """Callback to take action when smoothing slider changes"""
        geometry_dict = grating_geometry(sigma_slider.value, top_slider.value, 
                                         blaze_slider.value, new)
        bokeh_input = {key:np.array(geometry_dict[key]) for key in 
                       ['B_x_coordinates', 'B_y_coordinates', 
                        'C_x_coordinates', 'C_y_coordinates',
                       'D_x_coordinates', 'D_y_coordinates']}
        data_source.data = bokeh_input

    blaze_slider.on_change("value", update_upon_blaze)
    sigma_slider.on_change("value", update_upon_sigma)
    top_slider.on_change("value", update_upon_top)
    apex_slider.on_change("value", update_upon_apex)
    incidence_slider.on_change("value", update_upon_incidence)

    widgets_and_figures = layout([fig],
        [incidence_slider],
        [sigma_slider],
        [top_slider],
        [blaze_slider],
        [apex_slider],

    )
    doc.add_root(widgets_and_figures)

output_notebook(verbose=False, hide_banner=True)
show(create_interact_ui)

Nice! The interactive animation makes the groove shapes come to life.